In [108]:
import pandas as pd
df = pd.read_pickle("OPIF_04-06.pkl")

In [109]:
df['callsign'].value_counts()[:10]

WHNS       229
WMUR-TV    211
KRWF       180
KSAX       180
KFMB-TV    169
KTTC       159
WPTZ       156
KBJR-TV    152
KRII       152
KMPH-TV    147
Name: callsign, dtype: int64

In [110]:
select = 'KRWF'
dfquery = df[(df.callsign == select)]
file_urls = dfquery.file_url
file_names = dfquery.file_name

In [111]:
import os
path = select + ' Files/'
try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Creation of the directory KRWF Files/ failed


In [112]:
import requests, requests_cache, time
from IPython.display import clear_output
requests_cache.install_cache()

In [113]:
n = 1
errorCount = 0
for url, name in zip(file_urls, file_names):
    print("Downloading file {}/{}".format(n,len(file_urls)))
    
    r = requests.get(url, stream = True)
    if (r.status_code != 200): 
        print(url, r.text)
        errorCount += 1
        time.sleep(2)
    else: 
        with open(path + name + '.pdf', 'wb') as f:
            f.write(r.content)
        dfquery['file_path'] = path + name + '.pdf'
    if not getattr(r, 'from_cache', False):
        time.sleep(0.25)
    clear_output(wait = True)
    n += 1
print("Number of errors: ", errorCount)

Number of errors:  5


In [28]:
# import shutil
# shutil.make_archive(select + ' Files', 'zip', select + ' Files')

'C:\\Users\\Henry Weng\\PythonNotebooks\\KSAX Files.zip'

### We will test the PDF scraper on one file from the selected callsign.

In [22]:
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def scrape(pdf):
    output_string = StringIO()
    with open(pdf, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return output_string.getvalue()

#### Find the different search parameters for contracts and invoices.

#### Contracts

In [88]:
contract = scrape(path + file_names[353] + '.pdf')

In [89]:
contract[:300]

'Contract Agreement Between:\n\nPrint Date\n\n03/02/20\n\nPage\n\n1\n\nof\n\n2\n\nKSTP-TV, LLC\n3415 University Ave\nSaint Paul, MN  55114-2099\n(651)646-5555\n\nCONTRACT\n\nContract / Revision\n\nAlt Order #\n\n381617\n\n / \n\n09765140\n\nProduct\n\nJOE BIDEN PRESIDENT\n\nContract Dates\n\nEstimate #\n\n03/02/20 - 03/03/20\n\n7173\n\nAdvert'

In [90]:
i = contract.find('Contract Dates')
contract[i : i+50]

'Contract Dates\n\nEstimate #\n\n03/02/20 - 03/03/20\n\n7'

In [91]:
contract[i+28:i+47]

'03/02/20 - 03/03/20'

#### Invoices

In [92]:
inv = scrape(path + file_names[448] + '.pdf')

In [93]:
inv[:350]

'KSTP-TV, LLC\n3415 University Ave\nSaint Paul, MN  55114-2099\nMain:   (651)646-5555\nBilling: (651)642-4230\n\nINVOICE\n\nPage\n\n1\n\n3of\n\nProperty\n\nInvoice #\n\nKSTP_KSAX\n381617-1\n\nOrder #\n\n381617\n\nInvoice Date\n\n03/08/20\n\nAlt Order #\n\n09765140\n\nInvoice Month\n\nMarch 2020\n\nDeal #\n\nInvoice Period\n\n02/24/20 - 03/03/20\n\nFlight Dates\n\n03/02/20 - 03/03/20\n\nAdvertise'

In [94]:
i = inv.find('Flight Dates')
inv[i:i+50]

'Flight Dates\n\n03/02/20 - 03/03/20\n\nAdvertiser\n\nPro'

In [101]:
inv.find('Contract Dates')

-1

In [95]:
inv[i+14:i+33]

'03/02/20 - 03/03/20'

,candidate,entity_id,callsign,nielsen_dma_rank,create_ts,file_url,file_name,file_status,file_id,network_affiliation,active_ind,date,file_path
353,BIDEN,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-09 16:03:14.060000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,K5-Biden for President-ORD-381617,com_cpy,2d212039-478f-980e-03f9-d1056855b21c,ABC,Y,2020-03-09,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
354,BIDEN,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-09 16:03:14.010000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,45-Biden for President-ORD-381627,com_cpy,97845cba-5d05-cd84-c6e3-e9df46e56bae,ABC,Y,2020-03-09,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
355,BIDEN,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-09 16:03:14.084000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,K5-Biden for President-ORD-381617-REV1,com_cpy,92514b98-979c-4976-d6e6-e50542f4e5b9,ABC,Y,2020-03-09,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
446,BIDEN,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-31 20:46:23.108000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,45-Biden for President-INVOICE-381627-1,com_cpy,bb9063b3-77cd-716b-71a9-9dc295f9b8a9,ABC,Y,2020-03-31,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
447,BIDEN,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-31 20:46:23.131000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,45-Biden for President-ORDER-381627-CODED,com_cpy,2d108791-5042-a92f-4cbc-fe406eb7fd64,ABC,Y,2020-03-31,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,SANDERS,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-02-25 16:06:58.148000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,K5-Bernie 2020-ORD-380427-REV7,com_cpy,af6e49dd-9809-fa10-e62f-ac9def62cf7a,ABC,Y,2020-02-25,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
1616,SANDERS,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-09 16:05:07.921000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,K5-Bernie 2020-INV-380427-1,com_cpy,5c57a0e7-ee3d-905b-133a-af32761bcf07,ABC,Y,2020-03-09,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
1630,SANDERS,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-03-31 20:48:10.053000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,K5-Bernie 2020-ORDER-380427-CODED,com_cpy,31d26ee5-16d7-16dc-2bfb-b837555c71f7,ABC,Y,2020-03-31,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf
1634,SANDERS,35585,KRWF,MINNEAPOLIS-ST. PAUL,2020-04-02 20:19:19.195000+00:00,https://publicfiles.fcc.gov/api/manager/downlo...,K5-Bernie 2020-INVOICE-380427-2,com_cpy,3e27ea36-83a0-2513-ad8d-57f3f4c60924,ABC,Y,2020-04-02,KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf


#### Scrapes all the PDFs and returns as a list of strings.

In [114]:
dfquery['file_path'].value_counts()

KRWF Files/K5-Bernie 2020-INVOICE-380427-2.pdf    180
Name: file_path, dtype: int64

In [117]:
import re 
def getTotal(in_string):
    amtRegex = re.compile(r'(\$)(\d{1,3})(,\d{3})?(.\d{2})?')
    mo = re.findall(amtRegex, in_string)
    numbers = []
    for match in mo:
        entry = ''
        for j in match:
            if(j != '$'): entry += j
        numbers.append(entry)
    for i in range(len(numbers)):
        numbers[i] = numbers[i].replace(',', '')
    num_array = list(map(float, numbers))
    return max(num_array)

In [97]:
# import os, glob
# n = 1
# outputs = []
# ## Takes 2N seconds for N files to execute
# for filename in glob.glob(os.path.join(path, '*.pdf')):
#     print("Scraping file {}".format(n))
#     output_string = StringIO()
#     with open(filename, 'rb') as in_file: # open in readonly mode
#         parser = PDFParser(in_file)
#         doc = PDFDocument(parser)
#         rsrcmgr = PDFResourceManager()
#         device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
#         interpreter = PDFPageInterpreter(rsrcmgr, device)
#         for page in PDFPage.create_pages(doc):
#             interpreter.process_page(page)
#     outputs.append(output_string.getvalue())
#     clear_output(wait = True)
#     n += 1

In [121]:
# import pickle
# with open('output.pkl','wb') as f:
#     pickle.dump(outputs, f)